## RLHF（Reinforcement Learning from Human Feedback）

- **目标**：让模型更符合人类意图、更安全、更有用
- **核心思想**：
  - 用监督微调（SFT）教会模型基本的指令跟随
  - 用偏好数据训练奖励模型（RM），学会打分“更好/更差”的回答
  - 用强化学习（PPO）在奖励信号下优化策略，权衡质量、稳定性与多样性
- **关键组件**：指令数据、偏好数据（A/B 对比）、奖励模型、强化学习算法、KL 约束/参考策略
- **典型产物**：
  - SFT 模型（会做事）
  - RM 奖励模型（会打分）
  - PPO 后的对齐模型（做得更好）
  - DPO（取代 RM+PPO 的直接偏好优化）

### RLHF 的三阶段流程（工程化视角）

| 阶段 | 名称 | 作用 | 技术 |
|---|---|---|---|
| 1️⃣ | SFT（监督微调） | 教模型执行指令 | CrossEntropyLoss |
| 2️⃣ | 奖励模型（RM）训练 | 学会“什么样的回答更好” | Pairwise ranking (A > B) |
| 3️⃣ | PPO 强化优化 | 用奖励信号优化生成策略 | PPO 算法（Policy Gradient） |

### 实验设置：模型与数据集选择
- **模型**：Qwen2.5-1.5B-Instruct（中文指令能力强，小参数、易于 LoRA/QLoRA）
- **SFT 数据**：BelleGroup/train_0.5M_CN（中文指令-回答对，体量适中，可采样）
- **偏好数据（用于 DPO/RM）**：argilla/ultrafeedback-binarized-preferences（成对偏好，易直接用于 DPO）

## 环境安装

In [ ]:
# 安装需要的库
%env TOKENIZERS_PARALLELISM=false
%pip install -q torch torchvision torchaudio
%pip install -q "transformers>=4.44.0" "datasets>=2.18.0" "accelerate>=0.33.0" \
                "peft>=0.12.0" "trl>=0.9.6" "sentencepiece>=0.1.99" "safetensors>=0.4.5" \
                "huggingface_hub>=0.24.0" "modelscope>=1.14.0" "protobuf>=4.25.0" \
                "numpy>=1.24.0" "scipy>=1.10.0" "tiktoken>=0.7.0" 

# 仅在 CUDA 可用时安装 bitsandbytes（可选）
import torch
if torch.cuda.is_available():
    %pip install -q "bitsandbytes>=0.43.0"

# 打印关键版本，便于排查
import importlib.metadata as im
v = lambda n: (im.version(n) if n in {d.metadata['Name'] for d in im.distributions()} else 'N/A')
print("[Versions]",
      "torch=", v("torch"),
      "transformers=", v("transformers"),
      "datasets=", v("datasets"),
      "accelerate=", v("accelerate"),
      "peft=", v("peft"),
      "trl=", v("trl"),
      "modelscope=", v("modelscope"),
      "sentencepiece=", v("sentencepiece"),
      "bitsandbytes=", v("bitsandbytes"))


## 模型下载

In [21]:
# 下载Qwen/Qwen2.5-1.5B-Instruct
import torch
from modelscope.hub.snapshot_download import snapshot_download
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "Qwen/Qwen2.5-1.5B-Instruct"  # ModelScope 上的模型标识（公共可直接下载）

# 选择设备
device = "cuda" if torch.cuda.is_available() else ("mps" if torch.backends.mps.is_available() else "cpu")

# 通过 ModelScope 下载到本地缓存，然后用 Transformers 从本地目录加载
model_dir = snapshot_download(model_id)

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_dir, trust_remote_code=True)
model = model.to(device)

print(f"[Device] {device}")

# 简单自检
txt = "你好，简要介绍一下你自己。"
inputs = tokenizer(txt, return_tensors="pt").to(device)
with torch.inference_mode():
    out = model.generate(**inputs, max_new_tokens=64, do_sample=False)
print(tokenizer.decode(out[0], skip_special_tokens=True))

2025-11-01 23:44:09,291 - modelscope - INFO - Target directory already exists, skipping creation.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[Device] mps
你好，简要介绍一下你自己。 作为一个AI助手，我叫通义千问。我可以回答各种问题、提供信息和帮助您完成任务。有什么我能帮您的吗？


## 数据集下载

In [22]:
# 下载sft数据集和偏好数据集
from modelscope.hub.snapshot_download import snapshot_download

# 指定两个数据集名称（可按需修改）
sft_id = "AI-ModelScope/train_0.5M_CN"  # 中文数据集
pref_id_primary = "HuggingFaceH4/ultrafeedback_binarized" # 多语，英文为主，可筛出中文子集
 
# 仅使用 ModelScope 下载到本地缓存（不做回退）
sft_dir = snapshot_download(sft_id, repo_type="dataset")
pref_dir = snapshot_download(pref_id_primary, repo_type="dataset")

2025-11-01 23:44:37,906 - modelscope - INFO - Fetching dataset repo file list...


2025-11-01 23:44:39,194 - modelscope - INFO - Fetching dataset repo file list...


In [23]:
# 预览 SFT 和偏好数据集
from datasets import load_dataset, Dataset
import pandas as pd, os, json, glob

pd.set_option("display.max_colwidth", None)
# ========== 通用工具函数 ==========
def normalize_text(x):
    """将嵌套结构标准化为可读字符串"""
    if x is None:
        return None
    if isinstance(x, str):
        return x
    if isinstance(x, dict):
        return x.get("content") or x.get("text") or x.get("value") or str(x)
    if isinstance(x, (list, tuple)):
        parts = []
        for it in x:
            if isinstance(it, str):
                parts.append(it)
            elif isinstance(it, dict):
                role = it.get("role")
                content = it.get("content") or it.get("text") or it.get("value")
                if content:
                    parts.append((f"{role}: " if role else "") + str(content))
        return "\n---\n".join(parts)
    return str(x)
# ========== SFT 数据加载 ==========
print("📘 加载并预览 SFT 数据")
sft_preview = load_dataset(sft_dir, split="train[:2]")
sft_rows = []
for ex in sft_preview:
    sft_rows.append({
        "instruction": normalize_text(ex.get("instruction")),
        "input": normalize_text(ex.get("input")),
        "output": normalize_text(ex.get("output")),
        "instr_len": len(str(ex.get("instruction", ""))),
        "input_len": len(str(ex.get("input", ""))),
        "output_len": len(str(ex.get("output", ""))),
    })
print(f"[SFT] 预览 {len(sft_rows)} 条 / split=train[:2]")
display(pd.DataFrame(sft_rows))

# ========== 偏好数据加载（Primary） ==========
def load_pref_data(ds_dir, name="Primary"):
    """尝试多种方式加载偏好数据"""
    for split in ["train_prefs[:2]", "train[:2]"]:
        try:
            ds = load_dataset(ds_dir, split=split)
            return ds, f"{name} ({split.split('[')[0]})"
        except:
            continue
    # 尝试从 data 目录加载
    data_dir = os.path.join(ds_dir, "data")
    if not os.path.exists(data_dir):
        return None, None
    parquet = glob.glob(os.path.join(data_dir, "*.parquet"))
    if parquet:
        ds = Dataset.from_parquet(parquet[0]).select(range(2))
        return ds, f"{name} (Parquet)"
    jsonl = glob.glob(os.path.join(data_dir, "*.jsonl"))
    if jsonl:
        data = []
        with open(jsonl[0], "r", encoding="utf-8") as f:
            for i, line in enumerate(f):
                if i >= 2: break
                try: data.append(json.loads(line))
                except: pass
        return Dataset.from_list(data), f"{name} (JSONL)" if data else (None, None)
    return None, None

def format_pref_data(pref_ds, name="Primary"):
    if pref_ds is None:
        print(f"[Preference {name}] ❌ 无法加载")
        return
    rows = []
    for ex in pref_ds:
        row = {
            "prompt": normalize_text(ex.get("prompt") or ex.get("instruction") or ex.get("input")),
            "y_pos": normalize_text(ex.get("chosen") or ex.get("better_response") or ex.get("pos")),
            "y_neg": normalize_text(ex.get("rejected") or ex.get("worse_response") or ex.get("neg")),
        }
        row["prompt_len"] = len(str(row["prompt"] or ""))
        row["y_pos_len"] = len(str(row["y_pos"] or ""))
        row["y_neg_len"] = len(str(row["y_neg"] or ""))
        rows.append(row)
    print(f"[Preference {name}] 预览 {len(rows)} 条")
    display(pd.DataFrame(rows))

print("📗 加载并预览 Preference 数据")
pref_preview, pref_source = load_pref_data(pref_dir, "Primary")
format_pref_data(pref_preview, "Primary")

print(f"\n[Preference Primary] 数据集: {pref_dir.split('/')[-1]}")
if pref_preview and len(pref_preview) > 0:
    fields = list(pref_preview[0].keys())
    has_chosen = any(k in fields for k in ["chosen", "better_response"])
    has_rejected = any(k in fields for k in ["rejected", "worse_response"])
    print(f"  - 字段: {fields}")
    print(f"  - 支持: {'✅ RM/DPO' if has_chosen and has_rejected else '⚠️  部分支持'}")
else:
    print("  - 状态: ❌ 未加载")

📘 加载并预览 SFT 数据
[SFT] 预览 2 条 / split=train[:2]


,instruction,input,output,instr_len,input_len,output_len
0,给定一个英文句子，翻译成中文。\nI love to learn new things every day.\n,,我每天喜欢学习新事物。,54,0,11
1,给定一个文字输入，将其中的所有数字加1。\n“明天的会议在9点开始，记得准时到达。”\n,,“明天的会议在10点开始，记得准时到达。”,42,0,21


📗 加载并预览 Preference 数据
[Preference Primary] 预览 2 条


,prompt,y_pos,y_neg,prompt_len,y_pos_len,y_neg_len
0,how can i develop a habit of drawing daily,"user: how can i develop a habit of drawing daily\n---\nassistant: Developing a daily habit of drawing can be challenging but with consistent practice and a few tips, it can become an enjoyable and rewarding part of your daily routine. Here are some strategies to help you develop the habit of drawing daily:\n\n1. Set a specific time: Allocate a specific time of the day to draw. It could be in the morning, afternoon, or evening. Make drawing a part of your daily routine.\n2. Set a specific duration: Determine the amount of time you want to spend on drawing each day. It can be as little as 10 minutes or as long as an hour. Be consistent with the duration to help build the habit.\n3. Start small and simple: Don't try to create a masterpiece every day, start with simple and easy-to-do sketches. Focus on improving your skills gradually.\n4. Use a variety of tools and mediums: Experiment with different tools like pencils, pens, markers, and different mediums like paper, canvas, or digital apps to keep your drawing practice interesting and engaging.\n5. Take breaks and rest: Taking breaks and resting after some time of drawing can help you avoid burnout and stay motivated.\n6. Challenge yourself: Set challenges like drawing objects from memory or a specific subject to improve your skills and keep your drawing practice interesting.\n7. Track your progress: Keep a record of your daily drawing practice and track your progress. This can be a source of motivation and help you see how far you've come.\n\nRemember, developing a habit takes time and patience. Stay consistent with your drawing practice, be flexible and open to trying new things, and with time, you'll develop a habit of daily drawing that brings you joy and satisfaction.","user: how can i develop a habit of drawing daily\n---\nassistant: As an AI language model, I cannot personally develop habits for you. But, here are some tips for developing a habit of drawing daily:\n\n1. Start small: Start with simple drawings or doodles and gradually increase the complexity of your drawings.\n\n2. Set a schedule: Set a specific time every day to draw, and try to stick to it as much as possible.\n\n3. Make it fun: Don't pressure yourself to create masterpieces every time you draw. Make it a relaxing and enjoyable experience.\n\n4. Use resources: There are many drawing tutorials available online. Use resources like YouTube or online drawing courses to help you improve your skills.\n\n5. Surround yourself with inspiration: Expose yourself to a variety of art forms, such as paintings, illustrations, and photographs, to inspire and motivate you.\n\nRemember, everyone has their own creative style and pace. Just keep practicing and enjoying the process of drawing.",42,1737,977
1,how can I transform the getPosition method of antv/g's group in zrender?,"user: how can I transform the getPosition method of antv/g's group in zrender?\n---\nassistant: It is not recommended to modify built-in methods as it can lead to unexpected results and potential bugs. You may consider developing a new method or exploring other methods to achieve your desired outcome. Alternatively, you can search for other libraries or modules that offer similar functionalities or reach out to the library's support team for assistance.","user: how can I transform the getPosition method of antv/g's group in zrender?\n---\nassistant: Thank you for reaching out for assistance! I'm here to help you with your question. However, I must point out that the question itself may not be meaningful.\n\nThe `getPosition` method is a part of the AntV/G library, which is a gradient boosting framework. It is not clear what you mean by ""transforming"" this method, as it is not a functional programming concept. Additionally, the concept of ""zrender"" is not related to AntV/G.\n\nCould you please provide more context or clarify your question? I'd be happ


[Preference Primary] 数据集: ultrafeedback_binarized
  - 字段: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected']
  - 支持: ✅ RM/DPO


## 1️⃣ SFT（监督微调）
- **输入**：指令-回答对（高质量、人类书写/筛选）
- **目标**：让模型基本学会“按指令作答”
- **训练**：最小化交叉熵损失（参考常用指令数据集）
- **输出**：SFT 模型（作为后续 RM/PPO 的参考策略）

In [24]:
# 【加载sft数据集】 dataset
from datasets import Dataset
import os
import json

sft_path = os.path.expanduser(sft_dir)

# 检查目录内容
print("[文件列表]", os.listdir(sft_path))

# 尝试找到 JSON 或 JSONL 文件
json_files = [f for f in os.listdir(sft_path) if f.endswith(".json") or f.endswith(".jsonl")]
if not json_files:
    raise RuntimeError(f"在 {sft_path} 未找到 JSON/JSONL 文件，请手动查看文件结构。")

json_path = os.path.join(sft_path, json_files[0])
print(f"[加载文件] {json_path}")

# 加载 JSON 数据
data = []
with open(json_path, "r", encoding="utf-8") as f:
    for line in f:
        try:
            data.append(json.loads(line))
        except:
            pass

# 转换成 HuggingFace Dataset
sft_ds = Dataset.from_list(data)
print(f"[SFT] 成功加载 {len(sft_ds)} 条样本。")
print("[示例样本]", sft_ds[0])

[文件列表] ['.mdl', 'Belle_open_source_0.5M.jsonl', 'README.md', '.msc', '.gitattributes', '.mv', 'train_0.5M_CN.json']
[加载文件] /Users/arkin/.cache/modelscope/hub/datasets/AI-ModelScope/train_0___5M_CN/Belle_open_source_0.5M.jsonl
[SFT] 成功加载 519255 条样本。
[示例样本] {'instruction': '给定一个英文句子，翻译成中文。\nI love to learn new things every day.\n', 'input': '', 'output': '我每天喜欢学习新事物。'}


In [25]:
# 【数据集切分】按比例划分，比如 98% 训练、2% 验证
sft_split = sft_ds.train_test_split(test_size=0.02, seed=42)

# 重构数据格式：构造 prompt-completion 结构（符合 TRL 0.24 标准）
def _to_sft(example):
    instr = example.get("instruction", "")
    inp = example.get("input", "")
    output = example.get("output", None)
    # 使用空格结尾，避免分词边界问题
    prompt = (instr + ("\n" + inp if inp else "")).strip() + " "
    return {"prompt": prompt, "completion": output}

# 切分并格式化
train_ds = sft_split["train"].map(_to_sft, remove_columns=sft_split["train"].column_names)
eval_ds = sft_split["test"].map(_to_sft, remove_columns=sft_split["test"].column_names)

print("训练集样本数:", len(train_ds))
print("验证集样本数:", len(eval_ds))
print("[示例]", train_ds[0])


Map:   0%|          | 0/508869 [00:00<?, ? examples/s]

Map:   0%|          | 0/10386 [00:00<?, ? examples/s]

训练集样本数: 508869
验证集样本数: 10386
[示例] {'prompt': '查找一位著名音乐家的出生地和最著名的作品。\n音乐家：贝多芬 ', 'completion': '贝多芬的出生地是德国的波恩市。他最著名的作品包括第九交响曲、第五交响曲、月光奏鸣曲和费加罗的迷宫等。'}


In [26]:
# 【LoRA 配置与训练参数】
from peft import LoraConfig
from transformers import TrainingArguments

# 模型输入输出配置
base_model_or_dir = model_dir  # 复用 ModelScope 下载的模型目录
output_dir = "outputs/sft_qlora"
max_seq_length = 2048

use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

# bitsandbytes QLoRA 配置 
quantization_config = None
if use_cuda:
    try:
        from transformers import BitsAndBytesConfig
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
        print("[QLoRA] 使用 bitsandbytes 4-bit 量化加载模型")
    except Exception as e:
        print(f"[Warn] 未启用4bit量化：{e}")
else:
    print("[Info] 当前为非 CUDA 环境，使用常规精度加载")

# Tokenizer 初始化
if tokenizer is None:
    tokenizer = AutoTokenizer.from_pretrained(base_model_or_dir, use_fast=True, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

[Info] 当前为非 CUDA 环境，使用常规精度加载


In [ ]:
# LoRA 配置：低秩适配器参数
lora_config = LoraConfig(
    r=16,                    # 秩（rank），控制适配器矩阵的维度，越大容量越大但参数量更多（常用 8/16/32）
    lora_alpha=32,          # 缩放系数，与 r 成比例使用（通常 alpha=2*r）
    lora_dropout=0.05,      # 适配器层 dropout（防止过拟合，常用 0.05-0.1）
    bias="none",            # 是否训练 bias（"none"/"all"/"lora_only"）
    task_type="CAUSAL_LM",   # 任务类型：因果语言模型（生成任务）
)

# 训练参数：控制训练流程与优化
args = TrainingArguments(
    # 输出与保存
    output_dir=output_dir,              # 模型/日志输出目录
    save_steps=200,                     # 每 N 步保存一次 checkpoint
    save_total_limit=2,                 # 最多保留 N 个 checkpoint（避免占磁盘）
    save_safetensors=True,              # 使用 safetensors 格式保存（更快更安全）
    
    # 批大小与梯度
    per_device_train_batch_size=1,      # 每设备训练 batch 大小（显存受限时先用 1）
    per_device_eval_batch_size=1,       # 每设备验证 batch 大小
    gradient_accumulation_steps=8,      # 梯度累积步数（等效 batch = 1 * 8 = 8）
    
    # 训练轮次与学习率
    num_train_epochs=1,                 # 训练轮次数
    learning_rate=2e-4,                 # 初始学习率（LoRA 常用 1e-4 到 5e-4）
    lr_scheduler_type="cosine",         # 学习率调度器（余弦退火）
    warmup_ratio=0.03,                  # warmup 比例（前 3% 步数线性增长 LR）
    
    # 评估与日志
    eval_strategy="steps",              # 评估策略（"steps"/"epoch"/"no"）
    eval_steps=100,                     # 每 N 步评估一次
    logging_steps=10,                   # 每 N 步打印一次日志
    report_to=["none"],                 # 不向外部上报（可改为 ["wandb"] 等）
    
    # 模型检查点
    load_best_model_at_end=True,        # 训练结束加载最佳模型
    metric_for_best_model="eval_loss",  # 最佳模型指标
    greater_is_better=False,            # 该指标越小越好
    
    # 性能优化
    bf16=use_cuda,                      # CUDA 上用 bfloat16（算力与稳定性平衡）
    fp16=False,                         # 禁用 FP16（避免冲突）
    gradient_checkpointing=True,        # 梯度检查点（牺牲时间换显存）
)

print("训练参数配置完成")

In [ ]:
# 定义格式化函数：构造训练文本模板
def formatting_func(example):
    """将 prompt-response 对转换为模型输入文本（单条处理）"""
    prompt = str(example["prompt"]).strip()
    resp = str(example["response"]).strip()
    # 拼接格式：符合 Qwen 的模板
    text = f"<|user|>\n{prompt}\n<|assistant|>\n{resp}{tokenizer.eos_token}"
    return text

print("[格式化函数已定义]")
print("[示例]", formatting_func({"prompt": "你好", "response": "你好！有什么可以帮助您的吗？"}))

In [ ]:
# 训练器的构造
from trl import SFTTrainer

# 加载预训练模型（QLoRA 模式下自动应用量化）
model = AutoModelForCausalLM.from_pretrained(
    base_model_or_dir,
    trust_remote_code=True,
    quantization_config=quantization_config,
    device_map="auto" if use_cuda else None,
)

# 构造 SFT 训练器
trainer = SFTTrainer(
    model=model,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    peft_config=lora_config,
    args=args,
    # 不使用 formatting_func，直接用 prompt-completion 结构
)

In [ ]:
# 开始训练
trainer.train()

# === 保存适配器（LoRA 权重）===
adapter_dir = os.path.join(output_dir, "adapter")
trainer.model.save_pretrained(adapter_dir)
print(f"[Done] SFT + QLoRA 训练完成，LoRA 权重已保存至: {adapter_dir}")


## 2️⃣ 奖励模型（RM）训练
- **输入**：同一指令下成对回答（A、B），以及偏好标签（A > B）
- **目标**：学习“偏好评分函数” r(x, y)
- **训练**：Pairwise ranking（如 Bradley–Terry/Logistic loss）
- **输出**：能对任意回答打分的奖励模型

In [28]:
# 加载偏好数据集
from datasets import load_dataset, Dataset
import pandas as pd
import os
import json
import glob

if 'pref_dir' in globals():
    for split in ["train_prefs", "train"]:
        try:
            pref_ds = load_dataset(pref_dir, split=split)
            print(f"[RM] 从本地目录加载成功（{split}），共 {len(pref_ds)} 条样本")
            break
        except:
            continue
    # 如果还是失败，尝试从 data 目录加载 Parquet/JSONL
    if pref_ds is None:
        data_dir = os.path.join(pref_dir, "data") if os.path.exists(os.path.join(pref_dir, "data")) else pref_dir
        parquet_files = glob.glob(os.path.join(data_dir, "*.parquet"))
        if parquet_files:
            pref_ds = Dataset.from_parquet(parquet_files[0])
            print(f"[RM] 从 Parquet 加载成功，共 {len(pref_ds)} 条样本")
        else:
            jsonl_files = glob.glob(os.path.join(data_dir, "*.jsonl"))
            if jsonl_files:
                data = []
                with open(jsonl_files[0], "r", encoding="utf-8") as f:
                    for line in f:
                        try:
                            data.append(json.loads(line))
                        except:
                            pass
                pref_ds = Dataset.from_list(data)
                print(f"[RM] 从 JSONL 加载成功，共 {len(pref_ds)} 条样本")
if pref_ds is None:
    raise RuntimeError("无法加载偏好数据集，请检查数据路径")

# 预览数据结构
print(f"\n[RM] 数据集字段：{pref_ds[0].keys()}")
print(f"[RM] 示例样本：")
example = pref_ds[0]
for k, v in example.items():
    if isinstance(v, str) and len(v) > 100:
        print(f"  {k}: {v[:100]}...")
    else:
        print(f"  {k}: {v}")

[RM] 从本地目录加载成功（train_prefs），共 61135 条样本

[RM] 数据集字段：dict_keys(['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'])
[RM] 示例样本：
  prompt: how can i develop a habit of drawing daily
  prompt_id: 086b3e24f29b8956a01059f79c56db35d118a06fb6b844b095737d042795cd43
  chosen: [{'content': 'how can i develop a habit of drawing daily', 'role': 'user'}, {'content': "Developing a daily habit of drawing can be challenging but with consistent practice and a few tips, it can become an enjoyable and rewarding part of your daily routine. Here are some strategies to help you develop the habit of drawing daily:\n\n1. Set a specific time: Allocate a specific time of the day to draw. It could be in the morning, afternoon, or evening. Make drawing a part of your daily routine.\n2. Set a specific duration: Determine the amount of time you want to spend on drawing each day. It can be as little as 10 minutes or as long as an hour. Be consistent with the duration to help bu

In [29]:
# 转换为 RM 训练格式（prompt, chosen, rejected）
def normalize_text(x):
    """将嵌套结构标准化为可读字符串"""
    if x is None:
        return None
    if isinstance(x, str):
        return x
    if isinstance(x, dict):
        return x.get("content") or x.get("text") or x.get("value") or str(x)
    if isinstance(x, (list, tuple)):
        parts = []
        for it in x:
            if isinstance(it, str):
                parts.append(it)
            elif isinstance(it, dict):
                role = it.get("role")
                content = it.get("content") or it.get("text") or it.get("value")
                if content:
                    parts.append((f"{role}: " if role else "") + str(content))
        return "\n---\n".join(parts)
    return str(x)

def _to_rm_format(example):
    """将偏好数据转换为 RM 训练格式（prompt, chosen, rejected）"""
    # 尝试多种字段名
    prompt = normalize_text(
        example.get("prompt") or 
        example.get("instruction") or 
        example.get("input") or
        example.get("messages")
    )
    
    chosen = normalize_text(
        example.get("chosen") or 
        example.get("better_response") or 
        example.get("positive") or
        example.get("response_j")
    )
    
    rejected = normalize_text(
        example.get("rejected") or 
        example.get("worse_response") or 
        example.get("negative") or
        example.get("response_k")
    )
    
    # 处理 messages 格式（对话格式）
    if prompt is None and isinstance(example.get("messages"), list):
        msgs = example.get("messages", [])
        # 提取最后一个 user 消息作为 prompt，之前的作为 context
        user_msgs = [m.get("content", "") for m in msgs if m.get("role") == "user"]
        if user_msgs:
            prompt = user_msgs[-1]
    
    # 处理 chosen/rejected 如果是列表格式
    if isinstance(chosen, list):
        chosen = "\n---\n".join([normalize_text(c) for c in chosen])
    if isinstance(rejected, list):
        rejected = "\n---\n".join([normalize_text(r) for r in rejected])
    
    # 确保 prompt, chosen, rejected 都不为空
    if prompt is None or chosen is None or rejected is None:
        return None
    
    return {
        "prompt": str(prompt).strip(),
        "chosen": str(chosen).strip(),
        "rejected": str(rejected).strip(),
    }

# 转换数据格式
print("\n[RM] 开始转换数据格式...")
rm_ds = pref_ds.map(
    _to_rm_format,
    remove_columns=pref_ds.column_names,
    desc="转换为 RM 格式"
)

# 过滤掉 None 值（格式转换失败的数据）
rm_ds = rm_ds.filter(lambda x: x["prompt"] is not None and x["chosen"] is not None and x["rejected"] is not None)

print(f"[RM] 格式转换完成，有效样本数：{len(rm_ds)}")


[RM] 开始转换数据格式...
[RM] 格式转换完成，有效样本数：61135


In [30]:
# 数据切分：98% 训练，2% 验证
rm_split = rm_ds.train_test_split(test_size=0.02, seed=42)
rm_train_ds = rm_split["train"]
rm_eval_ds = rm_split["test"]

print(f"\n[RM] 数据切分完成：")
print(f"  训练集：{len(rm_train_ds)} 条")
print(f"  验证集：{len(rm_eval_ds)} 条")

# 预览训练集样本
print(f"\n[RM] 训练集示例：")
sample = rm_train_ds[0]
for k, v in sample.items():
    if isinstance(v, str) and len(v) > 150:
        print(f"  {k}: {v[:150]}...")
    else:
        print(f"  {k}: {v}")


[RM] 数据切分完成：
  训练集：59912 条
  验证集：1223 条

[RM] 训练集示例：
  prompt: cara membuat animasi splash screen menggunakan flutter
  chosen: user: cara membuat animasi splash screen menggunakan flutter
---
assistant: Untuk membuat animasi splash screen menggunakan Flutter, Anda harus mengik...
  rejected: user: cara membuat animasi splash screen menggunakan flutter
---
assistant: Saya tidak punya privilajek yang lengkap untuk menyembarkan cara membuat a...


In [ ]:
# 【RM 模型加载与配置】
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
from trl import RewardTrainer, RewardConfig
import torch

# 模型输入输出配置
rm_output_dir = "outputs/rm_qlora"
max_seq_length = 2048

use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

# 选择基础模型：优先使用 SFT 模型，否则使用基础模型
# 注意：RM 训练需要将因果语言模型转换为序列分类模型（添加奖励头）
if 'adapter_dir' in globals() and os.path.exists(adapter_dir):
    try:
        # 尝试从 SFT 模型加载（需要合并 LoRA 权重）
        from peft import PeftModel
        base_model_for_rm = AutoModelForCausalLM.from_pretrained(
            base_model_or_dir,
            trust_remote_code=True,
        )
        sft_model = PeftModel.from_pretrained(base_model_for_rm, adapter_dir)
        # 合并 LoRA 权重到基础模型
        merged_model = sft_model.merge_and_unload()
        rm_base_model_dir = None  # 标记使用合并后的模型
        print("[RM] 使用 SFT 模型作为基础（已合并 LoRA 权重）")
    except Exception as e:
        print(f"[RM] 无法加载 SFT 模型：{e}，使用基础模型")
        rm_base_model_dir = base_model_or_dir
else:
    rm_base_model_dir = base_model_or_dir
    print("[RM] 使用基础模型（未找到 SFT 模型）")

# bitsandbytes QLoRA 配置（可选）
rm_quantization_config = None
if use_cuda:
    try:
        rm_quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
        print("[RM] 使用 bitsandbytes 4-bit 量化加载模型")
    except Exception as e:
        print(f"[RM] 未启用4bit量化：{e}")
else:
    print("[RM] 当前为非 CUDA 环境，使用常规精度加载")

# Tokenizer 初始化（复用之前的 tokenizer）
if 'tokenizer' not in globals() or tokenizer is None:
    tokenizer = AutoTokenizer.from_pretrained(
        rm_base_model_dir if rm_base_model_dir else base_model_or_dir,
        use_fast=True,
        trust_remote_code=True
    )
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 加载奖励模型（将因果语言模型转换为序列分类模型）
# AutoModelForSequenceClassification 会自动添加分类头（奖励头）
if rm_base_model_dir:
    rm_model = AutoModelForSequenceClassification.from_pretrained(
        rm_base_model_dir,
        trust_remote_code=True,
        num_labels=1,  # 奖励分数是标量
        quantization_config=rm_quantization_config,
        device_map="auto" if use_cuda else None,
    )
else:
    # 如果使用合并后的 SFT 模型，需要从合并模型创建 RM 模型
    # 简化处理：直接使用基础模型（实际中可能需要更复杂的处理）
    rm_model = AutoModelForSequenceClassification.from_pretrained(
        base_model_or_dir,
        trust_remote_code=True,
        num_labels=1,
        quantization_config=rm_quantization_config,
        device_map="auto" if use_cuda else None,
    )

print("[RM] 奖励模型加载完成")

In [ ]:
# 【RM 数据格式化：构造 prompt-response 对】
def format_rm_prompt(prompt, response):
    """将 prompt 和 response 格式化为模型输入文本（符合 Qwen 模板）"""
    # 构造符合 Qwen 模板的文本格式
    text = f"<|user|>\n{prompt}\n<|assistant|>\n{response}"
    return text

def tokenize_rm_dataset(examples):
    """对 RM 数据集进行分词（处理 prompt+chosen 和 prompt+rejected 对）"""
    # 构造 chosen 和 rejected 的完整文本
    chosen_texts = [format_rm_prompt(p, c) for p, c in zip(examples["prompt"], examples["chosen"])]
    rejected_texts = [format_rm_prompt(p, r) for p, r in zip(examples["prompt"], examples["rejected"])]
    
    # 分词（chosen）- 不使用 return_tensors，因为 Dataset.map 期望返回普通列表
    chosen_tokenized = tokenizer(
        chosen_texts,
        max_length=max_seq_length,
        padding="max_length",
        truncation=True,
    )
    
    # 分词（rejected）- 不使用 return_tensors
    rejected_tokenized = tokenizer(
        rejected_texts,
        max_length=max_seq_length,
        padding="max_length",
        truncation=True,
    )
    
    # 返回格式化的数据（TRL RewardTrainer 期望的格式）
    return {
        "input_ids_chosen": chosen_tokenized["input_ids"],
        "attention_mask_chosen": chosen_tokenized["attention_mask"],
        "input_ids_rejected": rejected_tokenized["input_ids"],
        "attention_mask_rejected": rejected_tokenized["attention_mask"],
    }

# 对训练集和验证集进行分词
print("[RM] 开始对数据集进行分词...")
rm_train_tokenized = rm_train_ds.map(
    tokenize_rm_dataset,
    batched=True,
    desc="分词训练集"
)
rm_eval_tokenized = rm_eval_ds.map(
    tokenize_rm_dataset,
    batched=True,
    desc="分词验证集"
)

print(f"[RM] 分词完成：")
print(f"  训练集：{len(rm_train_tokenized)} 条")
print(f"  验证集：{len(rm_eval_tokenized)} 条")


In [ ]:
# 【RM LoRA 配置与训练参数】
# LoRA 配置（如果使用 LoRA 微调）
rm_lora_config = LoraConfig(
    r=16,                    # 秩（rank）
    lora_alpha=32,          # 缩放系数（通常 alpha=2*r）
    lora_dropout=0.05,      # 适配器层 dropout
    bias="none",            # 不训练 bias
    task_type="SEQ_CLS",    # 任务类型：序列分类（奖励模型）
)

# 训练参数：控制训练流程与优化
rm_args = RewardConfig(
    # 输出与保存
    output_dir=rm_output_dir,           # 模型/日志输出目录
    save_steps=500,                     # 每 N 步保存一次 checkpoint
    save_total_limit=2,                 # 最多保留 N 个 checkpoint
    save_safetensors=True,              # 使用 safetensors 格式保存
    
    # 批大小与梯度
    per_device_train_batch_size=1,      # 每设备训练 batch 大小
    per_device_eval_batch_size=1,       # 每设备验证 batch 大小
    gradient_accumulation_steps=8,      # 梯度累积步数（等效 batch = 1 * 8 = 8）
    
    # 训练轮次与学习率
    num_train_epochs=1,                 # 训练轮次数
    learning_rate=1e-5,                 # 初始学习率（RM 训练常用 1e-5 到 5e-5）
    lr_scheduler_type="cosine",         # 学习率调度器（余弦退火）
    warmup_ratio=0.03,                  # warmup 比例（前 3% 步数线性增长 LR）
    
    # 评估与日志
    eval_strategy="steps",              # 评估策略（"steps"/"epoch"/"no"）
    eval_steps=250,                     # 每 N 步评估一次
    logging_steps=50,                   # 每 N 步打印一次日志
    report_to=["none"],                 # 不向外部上报（可改为 ["wandb"] 等）
    
    # 模型检查点
    load_best_model_at_end=True,        # 训练结束加载最佳模型
    metric_for_best_model="eval_loss",  # 最佳模型指标
    greater_is_better=False,            # 该指标越小越好
    
    # 性能优化
    bf16=use_cuda,                      # CUDA 上用 bfloat16
    fp16=False,                         # 禁用 FP16
    gradient_checkpointing=True,        # 梯度检查点（牺牲时间换显存）
    
    # RM 特定参数
    max_length=max_seq_length,          # 最大序列长度
)

print("[RM] 训练参数配置完成")


In [ ]:
# 【构造 RM 训练器并开始训练】
# 如果使用 LoRA，应用 LoRA 配置到模型
if rm_quantization_config is not None or use_cuda:
    # 如果已经量化或使用 CUDA，应用 LoRA
    try:
        rm_model = get_peft_model(rm_model, rm_lora_config)
        print("[RM] LoRA 配置已应用到模型")
    except Exception as e:
        print(f"[RM] 应用 LoRA 时出错：{e}，继续使用全量微调")

# 构造 RewardTrainer
rm_trainer = RewardTrainer(
    model=rm_model,
    args=rm_args,
    tokenizer=tokenizer,
    train_dataset=rm_train_tokenized,
    eval_dataset=rm_eval_tokenized,
    peft_config=rm_lora_config if rm_quantization_config is None and not use_cuda else None,
)

print("[RM] 训练器构造完成，准备开始训练")


In [ ]:
# 开始训练
rm_trainer.train()

# 保存模型
rm_model_dir = os.path.join(rm_output_dir, "reward_model")
rm_trainer.model.save_pretrained(rm_model_dir)
tokenizer.save_pretrained(rm_model_dir)
print(f"[Done] RM + QLoRA 训练完成，模型已保存至: {rm_model_dir}")


## 3️⃣ PPO 强化优化
- **输入**：SFT 模型作为初始策略 \(\pi_\theta\)，奖励模型 r 作为奖励信号
- **目标**：在 KL 约束下最大化期望奖励，提升对齐度与有用性
- **训练**：PPO（剪切策略梯度），引入 KL 惩罚以保持与参考策略接近
- **输出**：PPO 后的对齐模型（更符合人类偏好）
- **实践要点**：高质量偏好数据与稳定的 KL 控制是成功关键；监控长度偏置、模式坍缩与过拟合。

In [ ]:
# 【PPO 训练准备：模型加载】
from transformers import AutoModelForCausalLM, AutoModelForSequenceClassification
from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
import torch

# PPO 输出配置
ppo_output_dir = "outputs/ppo_model"

use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

print("[PPO] 开始加载模型...")

# ========== 1. 加载策略模型（SFT 模型）作为初始策略 π_θ ==========
# PPO 需要策略模型（训练中的模型）和参考模型（用于 KL 约束）
if 'adapter_dir' in globals() and os.path.exists(adapter_dir):
    try:
        # 从 SFT 模型加载（带 LoRA 权重）
        from peft import PeftModel
        base_policy = AutoModelForCausalLM.from_pretrained(
            base_model_or_dir,
            trust_remote_code=True,
        )
        policy_model = PeftModel.from_pretrained(base_policy, adapter_dir)
        print("[PPO] 策略模型：使用 SFT 模型（带 LoRA 权重）")
        
        # 参考模型：使用合并后的基础模型（用于 KL 约束）
        reference_model = PeftModel.from_pretrained(base_policy, adapter_dir)
        # 创建参考模型的副本（冻结参数，不更新）
        reference_model.merge_and_unload()
        print("[PPO] 参考模型：使用 SFT 模型（已合并，用于 KL 约束）")
    except Exception as e:
        print(f"[PPO] 无法加载 SFT 模型：{e}，使用基础模型")
        policy_model = AutoModelForCausalLM.from_pretrained(
            base_model_or_dir,
            trust_remote_code=True,
        )
        reference_model = AutoModelForCausalLM.from_pretrained(
            base_model_or_dir,
            trust_remote_code=True,
        )
else:
    # 如果未找到 SFT 模型，使用基础模型
    policy_model = AutoModelForCausalLM.from_pretrained(
        base_model_or_dir,
        trust_remote_code=True,
    )
    reference_model = AutoModelForCausalLM.from_pretrained(
        base_model_or_dir,
        trust_remote_code=True,
    )
    print("[PPO] 策略模型和参考模型：使用基础模型")

# 将策略模型包装为带值函数的模型（PPO 需要）
# AutoModelForCausalLMWithValueHead 会在策略模型基础上添加值函数头（用于价值估计）
# 注意：如果 policy_model 是 PeftModel，需要先获取基础模型路径
try:
    if hasattr(policy_model, 'merge_and_unload'):
        # 对于 PeftModel，合并 LoRA 权重后创建 ValueHead 模型
        # 注意：合并后的模型会在内存中，需要保存到临时路径或直接使用基础模型
        print("[PPO] 检测到 PeftModel，从基础模型创建 ValueHead 模型（将使用 SFT 权重）")
        # 简化处理：直接从基础模型创建，SFT 权重将在后续加载
        ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(
            base_model_or_dir,
            trust_remote_code=True,
        )
    else:
        # 如果 policy_model 是普通模型，直接从基础模型创建
        ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(
            base_model_or_dir,
            trust_remote_code=True,
        )
    print("[PPO] 策略模型（带值函数头）加载完成")
except Exception as e:
    print(f"[PPO] 创建 ValueHead 模型失败：{e}")
    print("[PPO] 尝试直接从基础模型创建...")
    ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(
        base_model_or_dir,
        trust_remote_code=True,
    )

# ========== 2. 加载奖励模型（RM）作为奖励信号 ==========
if 'rm_model_dir' in globals() and os.path.exists(rm_model_dir):
    try:
        reward_model = AutoModelForSequenceClassification.from_pretrained(
            rm_model_dir,
            trust_remote_code=True,
            num_labels=1,  # 奖励分数是标量
        )
        print(f"[PPO] 奖励模型：从 {rm_model_dir} 加载成功")
    except Exception as e:
        print(f"[PPO] 无法加载奖励模型：{e}")
        reward_model = None
else:
    print("[PPO] 未找到奖励模型，需要在训练前先完成 RM 训练")
    reward_model = None

# Tokenizer 复用（如果已定义）
if 'tokenizer' not in globals() or tokenizer is None:
    tokenizer = AutoTokenizer.from_pretrained(
        base_model_or_dir,
        use_fast=True,
        trust_remote_code=True
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

print("[PPO] 模型加载完成")


In [ ]:
# 【PPO 数据准备：提示数据集】
# PPO 训练需要提示（prompts）数据集，模型会根据这些提示生成回答
# 然后奖励模型对生成的回答打分，PPO 根据奖励信号优化策略

from datasets import Dataset

# 方式 1：从 SFT 数据集中提取 prompts（推荐）
# 使用 SFT 训练集中的 prompts 作为 PPO 的输入
if 'train_ds' in globals() and len(train_ds) > 0:
    # 从 SFT 训练集中提取 prompts
    ppo_prompts = [ex["prompt"] for ex in train_ds.select(range(min(1000, len(train_ds))))]
    print(f"[PPO] 从 SFT 训练集提取 {len(ppo_prompts)} 个 prompts")
else:
    # 方式 2：创建简单的提示示例
    ppo_prompts = [
        "解释一下什么是机器学习。",
        "如何提高编程技能？",
        "什么是深度学习？",
        "如何学习 Python？",
        "介绍一下神经网络。",
    ]
    print(f"[PPO] 使用示例 prompts（共 {len(ppo_prompts)} 个）")

# 构造 PPO 提示数据集
ppo_dataset = Dataset.from_dict({"query": ppo_prompts})

print(f"[PPO] 提示数据集准备完成：{len(ppo_dataset)} 条 prompts")
print(f"[PPO] 示例 prompt: {ppo_dataset[0]['query']}")

# PPO 训练流程说明：
# 1. 策略模型根据 prompts 生成回答（多个候选回答）
# 2. 奖励模型对每个生成的回答打分
# 3. PPO 算法根据奖励信号优化策略，同时保持与参考模型的 KL 散度约束
# 4. 重复上述过程，直到策略收敛


## 🔍 RLHF 三阶段的权重存储详解

### 1️⃣ SFT（监督微调）阶段 - LoRA 权重存储

**存储方式**：仅保存 LoRA adapter 权重（轻量级）

```
SFT 训练后的模型结构：
┌─────────────────────────────────────┐
│   基础模型（Qwen2.5-1.5B）              │  ← 不保存，保持不变
│   ├── Embeddings                     │
│   ├── Transformer Layers (x24)       │
│   └── LM Head                         │
└─────────────────────────────────────┘
           ▲
           │ LoRA 适配器（小权重矩阵）
           │
┌─────────────────────────────────────┐
│   LoRA Adapters                     │  ← 保存这些
│   ├── LoRA_A (rank=16)              │    大小：~几MB
│   └── LoRA_B (rank=16)              │    参数量：~0.1% of 基础模型
└─────────────────────────────────────┘

保存路径：outputs/sft_qlora/adapter/
├── adapter_config.json              ← LoRA 配置
├── adapter_model.safetensors        ← LoRA 权重（轻量级）
└── tokenizer.json                   ← Tokenizer
```

**特点**：
- ✅ 仅保存少量参数（LoRA 权重，通常只有原模型的 0.1-1%）
- ✅ 文件小，便于存储和共享
- ✅ 需要基础模型才能使用（推理时需要合并）

**代码示例**：
```python
# SFT 训练后保存
adapter_dir = os.path.join(output_dir, "adapter")
trainer.model.save_pretrained(adapter_dir)  # 只保存 LoRA 权重

# 使用时的加载
base_model = AutoModelForCausalLM.from_pretrained(base_model_or_dir)
model = PeftModel.from_pretrained(base_model, adapter_dir)  # 加载 LoRA
```

---

### 2️⃣ RM（奖励模型）阶段 - 奖励头存储

**存储方式**：保存完整的奖励模型（基础模型 + 奖励头）

```
RM 训练后的模型结构：
┌─────────────────────────────────────┐
│   基础模型（可能是 SFT 合并后的）       │  ← 包含在内
│   ├── Embeddings                     │
│   ├── Transformer Layers             │
│   └── Hidden States                  │
└─────────────────────────────────────┘
           ▲
           │
┌─────────────────────────────────────┐
│   奖励头（Reward Head）                │  ← 这是新添加的
│   ├── Linear Layer (hidden_size)     │    关键组件！
│   └── Output: 1 (奖励分数标量)        │    大小：~几MB
└─────────────────────────────────────┘

保存路径：outputs/rm_qlora/reward_model/
├── config.json                       ← 模型配置（包含 num_labels=1）
├── model.safetensors                 ← 完整模型权重（基础模型 + 奖励头）
│                                       或 adapter_model.safetensors（如果用了 LoRA）
└── tokenizer.json                    ← Tokenizer
```

**特点**：
- ✅ 奖励头是新增的组件（Linear 层，输出维度为 1）
- ✅ 如果使用 LoRA，可能只保存 LoRA 权重；如果全量微调，保存完整模型
- ✅ 奖励头是关键：它学会了对回答打分（chosen > rejected）

**代码示例**：
```python
# RM 训练后保存
rm_model_dir = os.path.join(rm_output_dir, "reward_model")
rm_trainer.model.save_pretrained(rm_model_dir)  # 保存奖励模型（包含奖励头）

# 使用时的加载
reward_model = AutoModelForSequenceClassification.from_pretrained(
    rm_model_dir,
    num_labels=1,  # 奖励头输出维度为 1
)
```

---

### 3️⃣ PPO（强化优化）阶段 - 策略模型 + 值函数头存储

**存储方式**：保存策略模型权重 + 值函数头权重

```
PPO 训练后的模型结构：
┌─────────────────────────────────────┐
│   策略模型（Policy Model）             │  ← 可以是基础模型或 SFT+LoRA
│   ├── Embeddings                     │
│   ├── Transformer Layers             │
│   └── LM Head                         │
└─────────────────────────────────────┘
           ▲
           │ 两个头：策略头 + 值函数头
           │
┌─────────────────────────────────────┐
│   策略头（Policy Head）               │  ← 原有的（用于生成）
│   └── LM Head (vocab_size)          │
│                                      │
│   值函数头（Value Head）               │  ← 新添加的（用于 PPO）
│   ├── Linear Layer 1 (hidden_size)  │    关键组件！
│   └── Output: 1 (状态价值标量)       │    大小：~几MB
└─────────────────────────────────────┘

保存路径：outputs/ppo_model/
├── config.json                       ← 模型配置
├── pytorch_model.bin                  ← 策略模型权重
│   或 adapter_model.safetensors      ← 如果用了 LoRA（仅 LoRA 权重）
├── value_head/                        ← 值函数头（单独保存）
│   ├── config.json
│   └── pytorch_model.bin
└── tokenizer.json                     ← Tokenizer
```

**特点**：
- ✅ **值函数头（Value Head）**是 PPO 特有的组件
  - 用于估计状态价值（state value），计算优势函数（advantage）
  - 输出维度为 1（标量价值）
- ✅ 策略模型可以是基础模型或带 LoRA 的模型
- ✅ 如果使用 LoRA，可能只保存 LoRA 权重；值函数头通常全量保存

**代码示例**：
```python
# PPO 模型创建
ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(
    base_model_or_dir,  # 自动添加值函数头
)

# PPO 训练后保存
ppo_trainer.save_model(ppo_output_dir)  # 保存策略模型 + 值函数头

# 使用时的加载
ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(ppo_output_dir)
```

---

## 📊 三阶段权重存储对比

| 阶段 | 存储内容 | 存储大小 | 关键组件 | 能否独立使用 |
|------|---------|---------|---------|------------|
| **SFT** | LoRA Adapter 权重 | ~几MB（0.1-1% 参数量） | LoRA_A, LoRA_B | ❌ 需要基础模型 |
| **RM** | 基础模型 + 奖励头 | ~完整模型（或 LoRA） | 奖励头（Reward Head） | ✅ 可以独立使用 |
| **PPO** | 策略模型 + 值函数头 | ~完整模型（或 LoRA） | 值函数头（Value Head） | ✅ 可以独立使用 |

## 🔑 关键区别

### SFT vs RM vs PPO 的存储差异

1. **SFT**：
   - 只保存 LoRA 权重（轻量级）
   - 不修改基础模型
   - 推理时需要合并

2. **RM**：
   - 保存奖励头（新增的 Linear 层）
   - 如果全量微调，保存完整模型；如果 LoRA，保存 LoRA + 奖励头配置
   - 可以独立用于打分

3. **PPO**：
   - 保存值函数头（新增的组件，用于 RL）
   - 策略模型可以保持 LoRA 格式或全量保存
   - 可以独立用于生成（但值函数头只在训练时使用）


In [ ]:
# 【PPO 训练配置】
# PPO 训练参数：控制强化学习训练流程

ppo_config = PPOConfig(
    # 输出与保存
    output_dir=ppo_output_dir,              # 模型/日志输出目录
    save_steps=500,                         # 每 N 步保存一次 checkpoint
    save_total_limit=2,                     # 最多保留 N 个 checkpoint
    
    # 生成参数（策略模型生成回答时的参数）
    mini_batch_size=1,                     # PPO mini-batch 大小（每个 prompt 的处理批次）
    batch_size=8,                          # PPO batch 大小（收集经验的批次）
    gradient_accumulation_steps=1,         # 梯度累积步数
    
    # PPO 算法参数
    ppo_epochs=4,                           # PPO 更新轮次（每次收集经验后更新多少轮）
    learning_rate=1e-6,                     # PPO 学习率（通常较小，1e-6 到 1e-5）
    lr_scheduler_type="linear",             # 学习率调度器类型
    warmup_ratio=0.1,                       # warmup 比例
    
    # 奖励与 KL 约束
    init_kl_coef=0.1,                       # 初始 KL 惩罚系数（平衡奖励与 KL 散度）
    target=6.0,                             # KL 散度目标值（控制与参考模型的偏离程度）
    horizon=10000,                          # PPO horizon（奖励归一化参数）
    gamma=1.0,                              # 折扣因子（RL 中的未来奖励折扣）
    lam=0.95,                               # GAE lambda 参数（优势估计）
    
    # 生成参数（每个 prompt 生成多少个候选回答）
    num_padding_at_beginning=1,             # 填充位置（某些模型需要）
    
    # 训练控制
    max_grad_norm=1.0,                      # 梯度裁剪（防止梯度爆炸）
    report_to=["none"],                     # 不向外部上报（可改为 ["wandb"] 等）
    
    # 评估与日志
    log_with="none",                        # 日志记录工具（"wandb", "tensorboard" 等）
    logging_steps=10,                       # 每 N 步打印一次日志
    
    # 性能优化
    bf16=use_cuda,                          # CUDA 上用 bfloat16
    fp16=False,                             # 禁用 FP16
    
    # 序列长度
    max_length=max_seq_length,              # 最大序列长度
    max_new_tokens=512,                    # 每次生成的最大新 token 数
)

print("[PPO] 训练配置完成")
print(f"[PPO] 关键参数：")
print(f"  - PPO epochs: {ppo_config.ppo_epochs}（每次更新 {ppo_config.ppo_epochs} 轮）")
print(f"  - Batch size: {ppo_config.batch_size}（收集经验的批次大小）")
print(f"  - Mini batch size: {ppo_config.mini_batch_size}（PPO 更新的小批次）")
print(f"  - Learning rate: {ppo_config.learning_rate}（PPO 学习率）")
print(f"  - KL coefficient: {ppo_config.init_kl_coef}（KL 惩罚系数）")
print(f"  - Target KL: {ppo_config.target}（目标 KL 散度）")


In [ ]:
# 【PPO 训练器构造与训练流程说明】
from trl import PPOTrainer

print("[PPO] 构造 PPO 训练器...")

# 注意：在构造训练器之前，需要确保奖励模型已加载
if reward_model is None:
    print("[PPO] ⚠️  警告：奖励模型未加载，无法进行 PPO 训练")
    print("[PPO] 请先完成 RM 训练，确保 rm_model_dir 指向有效的奖励模型路径")
else:
    # 构造 PPO 训练器
    ppo_trainer = PPOTrainer(
        config=ppo_config,
        model=ppo_model,                    # 策略模型（带值函数头）
        ref_model=reference_model,          # 参考模型（用于 KL 约束）
        tokenizer=tokenizer,
        dataset=ppo_dataset,                # 提示数据集
    )
    
    print("[PPO] 训练器构造完成")
    print("\n" + "="*60)
    print("[PPO] PPO 训练流程说明：")
    print("="*60)
    print("""
    PPO（Proximal Policy Optimization）的训练流程：
    
    1️⃣ 【收集经验阶段】（Experience Collection）
       - 策略模型根据 prompts 生成回答
       - 奖励模型对生成的回答打分（reward）
       - 计算每个回答的优势（advantage）和价值（value）
       - 收集一个 batch 的经验（prompts, responses, rewards, advantages）
    
    2️⃣ 【PPO 更新阶段】（Policy Update）
       - 对收集的经验进行多轮 PPO 更新（ppo_epochs 轮）
       - 计算策略损失（policy loss）：最大化期望奖励
       - 计算价值损失（value loss）：价值函数的回归损失
       - 计算 KL 散度损失（KL penalty）：保持与参考策略的接近
       - 总损失 = policy_loss - value_loss + kl_penalty
       - 使用梯度上升优化策略参数
    
    3️⃣ 【KL 约束控制】（KL Divergence Control）
       - 动态调整 KL 惩罚系数（kl_coef）
       - 如果 KL 散度超过目标值，增加惩罚
       - 如果 KL 散度低于目标值，减少惩罚
       - 确保策略不会偏离参考模型太远（避免模式坍缩）
    
    4️⃣ 【重复迭代】
       - 重复步骤 1-3，直到策略收敛或达到最大步数
       - 定期保存 checkpoint
       - 监控奖励、KL 散度、生成质量等指标
    
    关键优势：
    - ✅ 稳定的策略更新（剪切策略梯度，避免大幅波动）
    - ✅ KL 约束防止模式坍缩
    - ✅ 支持在线学习（边生成边优化）
    - ✅ 适用于大规模语言模型
    
    注意事项：
    - ⚠️  需要高质量奖励模型（RM 的质量直接影响 PPO 效果）
    - ⚠️  KL 系数需要仔细调优（过大导致更新慢，过小导致偏离）
    - ⚠️  监控长度偏置（模型可能倾向于生成更长的回答以获得更高奖励）
    - ⚠️  防止过拟合（需要定期评估生成质量）
    """)
    print("="*60)
    
    # 构造奖励函数（用于 PPO 训练）
    def reward_function(samples):
        """奖励函数：使用奖励模型对生成的回答打分"""
        # 对每个生成的回答使用奖励模型打分
        inputs = tokenizer(
            samples,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=max_seq_length,
        ).to(reward_model.device)
        
        with torch.no_grad():
            # 奖励模型返回 logits（标量奖励分数）
            rewards = reward_model(**inputs).logits.squeeze(-1)
        
        return rewards
    
    print("\n[PPO] 奖励函数已定义（使用奖励模型打分）")
    print("[PPO] 训练器准备就绪，可以开始训练")
    
    print("\n" + "="*60)
    print("[PPO] 训练示例代码（注释掉，不实际执行）：")
    print("="*60)
    
    example_code = '''
    # 开始 PPO 训练循环
    generation_kwargs = {
        "max_new_tokens": 512,              # 生成的最大新 token 数
        "temperature": 1.0,                # 生成温度（控制随机性）
        "do_sample": True,                  # 是否采样
        "top_p": 0.95,                      # nucleus sampling
        "pad_token_id": tokenizer.eos_token_id,
    }
    
    # PPO 训练循环
    for epoch in range(1):  # 可以设置多个 epoch
        for batch in ppo_trainer.dataloader:
            # 1. 策略模型生成回答
            query_tensors = batch["input_ids"]
            response_tensors = ppo_trainer.generate(
                query_tensors,
                return_prompt=False,
                length_sampler=None,
                batch_size=ppo_config.batch_size,
                **generation_kwargs,
            )
            
            # 2. 提取生成的回答文本
            batch["response"] = [
                tokenizer.decode(r.squeeze(), skip_special_tokens=True)
                for r in response_tensors
            ]
            
            # 3. 计算奖励（使用奖励模型）
            texts = [
                q + r for q, r in zip(batch["query"], batch["response"])
            ]
            rewards = reward_function(texts)
            
            # 4. PPO 更新
            stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
            ppo_trainer.log_stats(stats, batch, rewards)
    
    # 保存最终模型
    ppo_trainer.save_model(ppo_output_dir)
    tokenizer.save_pretrained(ppo_output_dir)
    print(f"[Done] PPO 训练完成，模型已保存至: {ppo_output_dir}")
    '''
    
    print(example_code)
    print("="*60)


## DPO（Direct Preference Optimization）
- **定位**：作为（RM+PPO）的常见替代方案，用偏好对直接优化策略。
- **核心**：基于 \((x, y_{pos}, y_{neg})\) 提高 \(y_{pos}\) 概率、降低 \(y_{neg}\)，并以参考策略 \(\pi_{ref}\) 的对数概率差作隐式 KL 约束。
- **直观目标**：最小化 \(-\log \sigma\big(\beta[(\log \pi_\theta(y_{pos}|x) - \log \pi_\theta(y_{neg}|x)) - (\log \pi_{ref}(y_{pos}|x) - \log \pi_{ref}(y_{neg}|x))]\big)\)
- **优点**：流程简单、无奖励模型与 RL 回路、稳定易复现、吞吐高。
- **局限**：依赖高质量偏好数据；极端分布迁移下可控性较弱。

In [ ]:
# 【DPO 数据准备：复用偏好数据集】
# DPO 使用与 RM 相同的偏好数据格式（prompt, chosen, rejected）
# 可以直接复用之前加载的 RM 数据集

from datasets import Dataset

print("[DPO] 准备偏好数据...")

# 方式 1：直接复用 RM 训练数据（推荐）
if 'rm_train_ds' in globals() and len(rm_train_ds) > 0:
    # 复用 RM 的偏好数据集（已经是 prompt, chosen, rejected 格式）
    dpo_train_ds = rm_train_ds.select(range(min(5000, len(rm_train_ds))))  # 可以选择部分数据
    dpo_eval_ds = rm_eval_ds.select(range(min(200, len(rm_eval_ds))))
    print(f"[DPO] 复用 RM 训练数据：训练集 {len(dpo_train_ds)} 条，验证集 {len(dpo_eval_ds)} 条")
else:
    # 方式 2：重新加载偏好数据（如果 RM 数据不可用）
    print("[DPO] RM 数据不可用，尝试重新加载偏好数据...")
    if 'rm_ds' in globals():
        dpo_split = rm_ds.train_test_split(test_size=0.02, seed=42)
        dpo_train_ds = dpo_split["train"].select(range(min(5000, len(dpo_split["train"]))))
        dpo_eval_ds = dpo_split["test"].select(range(min(200, len(dpo_split["test"]))))
        print(f"[DPO] 从 rm_ds 加载：训练集 {len(dpo_train_ds)} 条，验证集 {len(dpo_eval_ds)} 条")
    else:
        # 方式 3：使用示例数据（仅用于演示）
        print("[DPO] ⚠️  警告：使用示例数据，实际训练应使用真实偏好数据")
        dpo_examples = [
            {
                "prompt": "解释一下什么是机器学习。",
                "chosen": "机器学习是人工智能的一个分支，通过算法让计算机从数据中学习规律，无需明确编程就能做出预测或决策。",
                "rejected": "机器学习就是一种编程方式。"
            },
            {
                "prompt": "如何学习 Python？",
                "chosen": "学习 Python 可以从基础语法开始，然后逐步学习数据结构、面向对象编程，并通过实际项目练习提升。",
                "rejected": "Python 很简单，随便学学就行。"
            },
        ]
        dpo_train_ds = Dataset.from_list(dpo_examples)
        dpo_eval_ds = Dataset.from_list(dpo_examples[:1])

print(f"\n[DPO] 数据准备完成：")
print(f"  训练集：{len(dpo_train_ds)} 条")
print(f"  验证集：{len(dpo_eval_ds)} 条")
print(f"\n[DPO] 示例数据：")
sample = dpo_train_ds[0]
for k, v in sample.items():
    if isinstance(v, str) and len(v) > 100:
        print(f"  {k}: {v[:100]}...")
    else:
        print(f"  {k}: {v}")


In [ ]:
# 【DPO 模型加载：策略模型和参考模型】
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, PeftModel
from trl import DPOTrainer, DPOConfig
import torch

# DPO 输出配置
dpo_output_dir = "outputs/dpo_qlora"
max_seq_length = 2048

use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

print("[DPO] 开始加载模型...")

# ========== 1. 加载策略模型（通常是 SFT 模型）==========
# DPO 需要策略模型（训练中的模型）和参考模型（用于 KL 约束，冻结参数）
if 'adapter_dir' in globals() and os.path.exists(adapter_dir):
    try:
        # 从 SFT 模型加载（带 LoRA 权重）
        base_policy = AutoModelForCausalLM.from_pretrained(
            base_model_or_dir,
            trust_remote_code=True,
        )
        dpo_policy_model = PeftModel.from_pretrained(base_policy, adapter_dir)
        print("[DPO] 策略模型：使用 SFT 模型（带 LoRA 权重）")
    except Exception as e:
        print(f"[DPO] 无法加载 SFT 模型：{e}，使用基础模型")
        dpo_policy_model = AutoModelForCausalLM.from_pretrained(
            base_model_or_dir,
            trust_remote_code=True,
        )
else:
    # 如果未找到 SFT 模型，使用基础模型
    dpo_policy_model = AutoModelForCausalLM.from_pretrained(
        base_model_or_dir,
        trust_remote_code=True,
    )
    print("[DPO] 策略模型：使用基础模型（未找到 SFT 模型）")

# ========== 2. 加载参考模型（用于 KL 约束，冻结参数）==========
# 参考模型通常是 SFT 模型的副本，但在训练过程中冻结参数
if 'adapter_dir' in globals() and os.path.exists(adapter_dir):
    try:
        # 加载参考模型（与策略模型相同，但冻结参数）
        base_ref = AutoModelForCausalLM.from_pretrained(
            base_model_or_dir,
            trust_remote_code=True,
        )
        # 对于 LoRA 模型，可以加载相同的 adapter，然后在训练器中冻结
        dpo_ref_model = PeftModel.from_pretrained(base_ref, adapter_dir)
        # 冻结参考模型的参数（DPOTrainer 会自动处理）
        print("[DPO] 参考模型：使用 SFT 模型（带 LoRA 权重，训练时冻结）")
    except Exception as e:
        print(f"[DPO] 无法加载 SFT 模型作为参考：{e}，使用基础模型")
        dpo_ref_model = AutoModelForCausalLM.from_pretrained(
            base_model_or_dir,
            trust_remote_code=True,
        )
else:
    # 如果未找到 SFT 模型，使用基础模型
    dpo_ref_model = AutoModelForCausalLM.from_pretrained(
        base_model_or_dir,
        trust_remote_code=True,
    )
    print("[DPO] 参考模型：使用基础模型（未找到 SFT 模型）")

# Tokenizer 初始化（如果之前未定义）
if 'tokenizer' not in globals() or tokenizer is None:
    tokenizer = AutoTokenizer.from_pretrained(
        base_model_or_dir,
        use_fast=True,
        trust_remote_code=True
    )
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

print("[DPO] 模型加载完成")
print(f"[DPO] 策略模型：{type(dpo_policy_model).__name__}")
print(f"[DPO] 参考模型：{type(dpo_ref_model).__name__}")


In [ ]:
# 【DPO LoRA 配置与训练参数】
# DPO 可以使用 LoRA 微调，也可以全量微调

# LoRA 配置（如果需要使用 LoRA）
dpo_lora_config = None
if True:  # 默认使用 LoRA（更节省显存）
    dpo_lora_config = LoraConfig(
        r=16,                    # 秩（rank）
        lora_alpha=32,          # 缩放系数（通常 alpha=2*r）
        lora_dropout=0.05,      # 适配器层 dropout
        bias="none",            # 不训练 bias
        task_type="CAUSAL_LM",  # 任务类型：因果语言模型
    )
    print("[DPO] 使用 LoRA 配置（r=16, alpha=32）")
else:
    print("[DPO] 使用全量微调（不使用 LoRA）")

# bitsandbytes QLoRA 配置（可选）
dpo_quantization_config = None
if use_cuda:
    try:
        dpo_quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
        print("[DPO] 使用 bitsandbytes 4-bit 量化加载模型")
    except Exception as e:
        print(f"[DPO] 未启用4bit量化：{e}")
else:
    print("[DPO] 当前为非 CUDA 环境，使用常规精度加载")

# DPO 训练参数：控制训练流程与优化
dpo_args = TrainingArguments(
    # 输出与保存
    output_dir=dpo_output_dir,              # 模型/日志输出目录
    save_steps=500,                          # 每 N 步保存一次 checkpoint
    save_total_limit=2,                      # 最多保留 N 个 checkpoint
    save_safetensors=True,                   # 使用 safetensors 格式保存
    
    # 批大小与梯度
    per_device_train_batch_size=2,          # 每设备训练 batch 大小（DPO 需要处理 chosen/rejected 对）
    per_device_eval_batch_size=2,           # 每设备验证 batch 大小
    gradient_accumulation_steps=4,          # 梯度累积步数（等效 batch = 2 * 4 = 8）
    
    # 训练轮次与学习率
    num_train_epochs=1,                      # 训练轮次数
    learning_rate=1e-5,                     # 初始学习率（DPO 常用 1e-5 到 5e-5）
    lr_scheduler_type="cosine",             # 学习率调度器（余弦退火）
    warmup_ratio=0.1,                       # warmup 比例（前 10% 步数线性增长 LR）
    
    # 评估与日志
    eval_strategy="steps",                  # 评估策略（"steps"/"epoch"/"no"）
    eval_steps=250,                         # 每 N 步评估一次
    logging_steps=50,                       # 每 N 步打印一次日志
    report_to=["none"],                     # 不向外部上报（可改为 ["wandb"] 等）
    
    # 模型检查点
    load_best_model_at_end=True,            # 训练结束加载最佳模型
    metric_for_best_model="eval_loss",     # 最佳模型指标
    greater_is_better=False,                # 该指标越小越好
    
    # 性能优化
    bf16=use_cuda,                          # CUDA 上用 bfloat16
    fp16=False,                             # 禁用 FP16
    gradient_checkpointing=True,           # 梯度检查点（牺牲时间换显存）
)

# DPO 特定配置（beta 温度参数）
dpo_config = DPOConfig(
    beta=0.1,                               # 温度参数（控制 KL 约束强度，常用 0.1-0.5）
    loss_type="sigmoid",                    # 损失类型（"sigmoid" 或 "hinge"）
    label_smoothing=0.0,                    # 标签平滑（0.0 表示不平滑）
    reference_free=False,                   # 是否不使用参考模型（False 表示使用）
)

print("[DPO] 训练参数配置完成")
print(f"[DPO] 关键参数：")
print(f"  - Learning rate: {dpo_args.learning_rate}（DPO 学习率）")
print(f"  - Beta (β): {dpo_config.beta}（KL 约束强度，越大越接近参考模型）")
print(f"  - Batch size: {dpo_args.per_device_train_batch_size}（每设备批次大小）")
print(f"  - Gradient accumulation: {dpo_args.gradient_accumulation_steps}（梯度累积步数）")


In [ ]:
# 【DPO 训练器构造与训练】
from trl import DPOTrainer

print("[DPO] 构造 DPO 训练器...")

# 如果策略模型使用了量化，需要应用 LoRA（如果使用）
if dpo_quantization_config is not None or use_cuda:
    # 如果已经量化或使用 CUDA，尝试应用 LoRA
    try:
        if dpo_lora_config is not None:
            from peft import get_peft_model
            # 注意：如果策略模型已经是 PeftModel，需要特殊处理
            if not isinstance(dpo_policy_model, PeftModel):
                dpo_policy_model = get_peft_model(dpo_policy_model, dpo_lora_config)
                print("[DPO] LoRA 配置已应用到策略模型")
    except Exception as e:
        print(f"[DPO] 应用 LoRA 时出错：{e}，继续使用当前配置")

# 构造 DPO 训练器
dpo_trainer = DPOTrainer(
    model=dpo_policy_model,                  # 策略模型（要优化的模型）
    ref_model=dpo_ref_model,                 # 参考模型（用于 KL 约束，冻结参数）
    args=dpo_args,                           # 训练参数
    beta=dpo_config.beta,                    # 温度参数（KL 约束强度）
    train_dataset=dpo_train_ds,             # 训练数据集（prompt, chosen, rejected）
    eval_dataset=dpo_eval_ds,                # 验证数据集
    tokenizer=tokenizer,                     # Tokenizer
    peft_config=dpo_lora_config if dpo_quantization_config is None and not use_cuda else None,  # LoRA 配置
    max_length=max_seq_length,              # 最大序列长度
    max_target_length=max_seq_length,        # 最大目标长度
    loss_type=dpo_config.loss_type,          # 损失类型（"sigmoid" 或 "hinge"）
    label_smoothing=dpo_config.label_smoothing,  # 标签平滑
)

print("[DPO] 训练器构造完成")

print("\n" + "="*60)
print("[DPO] DPO 训练流程说明：")
print("="*60)
print("""
DPO（Direct Preference Optimization）的训练流程：

1️⃣ 【数据准备】
   - 准备偏好数据（prompt, chosen, rejected）
   - chosen：更好的回答
   - rejected：更差的回答

2️⃣ 【模型准备】
   - 策略模型（π_θ）：要优化的模型（通常是 SFT 模型）
   - 参考模型（π_ref）：用于 KL 约束的基准模型（冻结参数）

3️⃣ 【DPO 训练】
   - 计算策略模型对 chosen/rejected 的对数概率
   - 计算参考模型对 chosen/rejected 的对数概率
   - 计算 DPO 损失：
     L = -log σ(β[(log π_θ(chosen|x) - log π_θ(rejected|x)) - 
                  (log π_ref(chosen|x) - log π_ref(rejected|x))])
   - 优化策略模型，最大化 chosen 概率，最小化 rejected 概率
   - 通过隐式 KL 约束防止偏离参考模型

4️⃣ 【保存模型】
   - 保存优化后的策略模型（可以是 LoRA 权重）

关键优势：
- ✅ 无需训练奖励模型（RM）
- ✅ 无需强化学习（PPO）
- ✅ 训练简单稳定（监督学习）
- ✅ 训练速度快（直接优化）
- ✅ 易复现（确定性训练）

关键参数：
- β (beta)：温度参数，控制 KL 约束强度（常用 0.1-0.5）
  - β 越大 → 更接近参考模型（保守）
  - β 越小 → 更偏离参考模型（激进）
""")
print("="*60)

# 开始训练（注释掉，用户可以选择是否执行）
print("\n[DPO] 准备开始训练...")
print("[DPO] 要开始训练，请取消下面代码的注释")

# 训练代码（注释掉，不实际执行）
# dpo_trainer.train()

# 保存模型（注释掉）
# dpo_model_dir = os.path.join(dpo_output_dir, "dpo_model")
# dpo_trainer.model.save_pretrained(dpo_model_dir)
# tokenizer.save_pretrained(dpo_model_dir)
# print(f"[Done] DPO 训练完成，模型已保存至: {dpo_model_dir}")

print("\n[DPO] 训练代码已准备，可以开始训练")


## 📊 DPO vs RM+PPO 完整对比

### 🔄 流程对比

```
传统 RLHF（RM + PPO）流程：
┌─────────────────────────────────────────┐
│  1️⃣ SFT（监督微调）                        │
│     ↓                                    │
│  2️⃣ RM（奖励模型训练）                     │
│     ├── 训练奖励头                        │
│     └── 学会打分（chosen > rejected）      │
│     ↓                                    │
│  3️⃣ PPO（强化优化）                        │
│     ├── 策略模型生成回答                   │
│     ├── 奖励模型打分                       │
│     ├── PPO 优化策略                      │
│     └── 值函数头（用于优势估计）            │
└─────────────────────────────────────────┘

DPO 流程：
┌─────────────────────────────────────────┐
│  1️⃣ SFT（监督微调）                        │
│     ↓                                    │
│  2️⃣ DPO（直接偏好优化）                    │
│     ├── 直接用偏好数据训练                │
│     ├── 最大化 chosen 概率               │
│     ├── 最小化 rejected 概率              │
│     └── 隐式 KL 约束（通过参考模型）       │
└─────────────────────────────────────────┘
```

### 🎯 关键区别

| 特性 | **RM + PPO** | **DPO** |
|------|------------|---------|
| **训练阶段** | 3 阶段（SFT → RM → PPO） | 2 阶段（SFT → DPO） |
| **需要奖励模型** | ✅ 是 | ❌ 否 |
| **需要强化学习** | ✅ 是（PPO） | ❌ 否 |
| **需要值函数头** | ✅ 是（用于 PPO） | ❌ 否 |
| **KL 约束方式** | 显式 KL 惩罚 | 隐式 KL 约束（通过参考模型） |
| **训练复杂度** | 高（RL 回路） | 低（监督学习） |
| **训练稳定性** | 中等（RL 不稳定） | 高（监督学习） |
| **训练速度** | 慢（生成+打分循环） | 快（直接优化） |
| **数据需求** | Prompts（生成时用） | 偏好对（chosen/rejected） |
| **资源需求** | 高（需要 RM） | 低（无需 RM） |
| **适用场景** | 需要灵活奖励控制 | 固定偏好数据 |

### 📦 权重存储对比

| 阶段 | **RM + PPO** | **DPO** |
|------|------------|---------|
| **SFT** | LoRA 权重 | LoRA 权重 |
| **RM** | 基础模型 + 奖励头 | ❌ 不需要 |
| **PPO** | 策略模型 + 值函数头 | ❌ 不需要 |
| **DPO** | ❌ 不需要 | 策略模型（可以是 LoRA） |

### ✅ 何时使用 DPO

**适合使用 DPO 的场景**：
- ✅ 有高质量偏好数据（chosen/rejected 对）
- ✅ 不需要动态奖励调整
- ✅ 追求训练简单性和稳定性
- ✅ 资源受限（不想训练奖励模型）
- ✅ 需要快速迭代

**适合使用 RM+PPO 的场景**：
- ✅ 需要动态奖励调整（在线学习）
- ✅ 需要灵活的奖励控制
- ✅ 偏好数据较少，但有很多 prompts
- ✅ 需要精细的 RL 控制

### 🔑 DPO 的关键参数

1. **beta (β)**：温度参数，控制 KL 约束强度
   - **范围**：0.1 - 0.5（常用）
   - **β 越大**：更保守，更接近参考模型
   - **β 越小**：更激进，更容易偏离参考模型
   - **推荐值**：0.1（默认），根据实际效果调整

2. **参考模型**：用于 KL 约束的基准模型
   - **通常是**：SFT 模型（冻结参数）
   - **作用**：防止策略模型偏离太远
   - **关键**：参考模型必须与策略模型初始化相同

3. **损失类型**：
   - **sigmoid**：标准的 sigmoid 损失（推荐）
   - **hinge**：hinge 损失（在某些场景下更稳定）

### 💡 DPO 实践建议

1. **数据质量**：偏好数据质量直接影响 DPO 效果
2. **beta 调优**：根据实际情况调整 beta 参数
3. **参考模型**：确保参考模型与策略模型初始化一致
4. **评估指标**：监控 chosen/rejected 的概率差异
5. **防止过拟合**：使用验证集定期评估
